# Import Libraries

In [1]:
import numpy as np

In [ ]:
data = 

# Define column names

In [ ]:
id = ['id']

redshift = ['redshift']

des_filters_cigale = ['ctio_decam.g', 'ctio_decam.r', 'ctio_decam.i', 'ctio_decam.z', 'ctio_decam.Y']
des_errors_cigale = ['ctio_decam.g_err', 'ctio_decam.r_err', 'ctio_decam.i_err', 'ctio_decam.z_err', 'ctio_decam.Y_err']

viking_filters_cigale = ['paranal.vircam.Z', 'paranal.vircam.Y', 'paranal.vircam.J', 'paranal.vircam.H', 'paranal.vircam.Ks']
viking_errors_cigale = ['paranal.vircam.Z_err', 'paranal.vircam.Y_err', 'paranal.vircam.J_err', 'paranal.vircam.H_err', 'paranal.vircam.Ks_err']

catwise_filters_cigale = ['wise.W1', 'wise.W2']
catwise_errors_cigale = ['wise.W1_err', 'wise.W2_err']

In [ ]:
des_cols_dict = {
    'DES_mag_auto_g_dered': 'ctio_decam.g',
    'DES_mag_auto_r_dered': 'ctio_decam.r',
    'DES_mag_auto_i_dered': 'ctio_decam.i',
    'DES_mag_auto_z_dered': 'ctio_decam.z',
    'DES_mag_auto_y_dered': 'ctio_decam.Y',
    'DES_magerr_auto_g' : 'ctio_decam.g_err',
    'DES_magerr_auto_r' : 'ctio_decam.r_err',
    'DES_magerr_auto_i' : 'ctio_decam.i_err',
    'DES_magerr_auto_z' : 'ctio_decam.z_err',
    'DES_magerr_auto_y' : 'ctio_decam.Y_err',
}

viking_cols_dict = {
    'VKG_zAperMag3_dered': 'paranal.vircam.Z',
    'VKG_yAperMag3_dered': 'paranal.vircam.Y',
    'VKG_jAperMag3_dered': 'paranal.vircam.J',
    'VKG_hAperMag3_dered': 'paranal.vircam.H',
    'VKG_ksAperMag3_dered': 'paranal.vircam.Ks',
    'VKG_zAperMag3Err_dered': 'paranal.vircam.Z_err',
    'VKG_yAperMag3Err_dered': 'paranal.vircam.Y_err',
    'VKG_jAperMag3Err_dered': 'paranal.vircam.J_err',
    'VKG_hAperMag3Err_dered': 'paranal.vircam.H_err',
    'VKG_ksAperMag3Err_dered': 'paranal.vircam.Ks_err',

}

catwise_cols_dict = {
    'CAT_w1mpro': 'wise.W1',
    'CAT_w2mpro': 'wise.W2',
}

# Add custom DES filters

### Normalize the transmission

In [2]:
g_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.g.dat'
i_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.i.dat'
r_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.r.dat'
z_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.z.dat'
y_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.Y.dat'

In [3]:
def normalize_filter_data(input_filename, output_filename):
    """
    Normalizes the second column (transmission values) of a .dat file.

    Parameters:
    - input_filename (str): Path to the input .dat file.
    - output_filename (str): Path where the normalized data will be saved.

    The input file should contain two columns: wavelength (first column) and transmission (second column).
    """
    # Initialize lists to store data
    lambda_values = []
    transmission_values = []

    # Read the data from the .dat file
    with open(input_filename, 'r') as f:
        for line in f:
            # Skip comments or empty lines
            if line.startswith('#') or line.strip() == '':
                continue
            
            # Split the line into wavelength and transmission
            columns = line.split()
            lambda_values.append(float(columns[0]))  # Wavelength (first column)
            transmission_values.append(float(columns[1]))  # Transmission (second column)

    # Convert the lists to numpy arrays for easier processing
    lambda_values = np.array(lambda_values)
    transmission_values = np.array(transmission_values)

    # Normalize the transmission values (second column)
    max_transmission = np.max(transmission_values)
    normalized_transmission = transmission_values / max_transmission

    # Save the normalized data to the output .dat file
    with open(output_filename, 'w') as f:
        # Optionally add a header
        f.write("# Wavelength (Å) and Normalized Transmission Values\n")
        
        # Write the wavelength and the normalized transmission
        for lambda_val, norm_trans in zip(lambda_values, normalized_transmission):
            f.write(f"{lambda_val} {norm_trans}\n")

    print(f"Normalization complete. Output saved to {output_filename}.")

In [4]:
normalize_filter_data(g_filter_path, g_filter_path)
normalize_filter_data(i_filter_path, i_filter_path)
normalize_filter_data(r_filter_path, r_filter_path)
normalize_filter_data(y_filter_path, y_filter_path)
normalize_filter_data(z_filter_path, z_filter_path)


Normalization complete. Output saved to /data/mfonseca/survey_data/DES_data/CTIO_DECam.g.dat.
Normalization complete. Output saved to /data/mfonseca/survey_data/DES_data/CTIO_DECam.i.dat.
Normalization complete. Output saved to /data/mfonseca/survey_data/DES_data/CTIO_DECam.r.dat.
Normalization complete. Output saved to /data/mfonseca/survey_data/DES_data/CTIO_DECam.Y.dat.
Normalization complete. Output saved to /data/mfonseca/survey_data/DES_data/CTIO_DECam.z.dat.


# Convert magnitudes to fluxes

In [ ]:
def mag_to_flux_mJy(mag, zero_point):
    """
    Convert magnitude to flux using the formula:
    flux_mJy = 3631 * 10 ** (-0.4 * mag)

    Args:
        mag (float or array-like): Magnitude value(s) to convert.

    Returns:
        float or array-like: Corresponding flux value(s) in mJy.

    """
    return 3631 * 10 ** (-0.4 * mag)

# Create a catalog for CIGALE

Creates an ASCII file for cigale 